In [11]:
import sys
sys.path.append('../../../')

import pandas as pd

import keras

from utils.predictor import PredictorToolkit
from utils.dataset import TreadingDataset
from utils.strategy import TradeStrategy
from utils.backtester import Backtester
from utils.field_processing import create_ts_from_df
import warnings

warnings.filterwarnings('ignore')

In [12]:
SEQUENCE_LENGTH = 9
SHIFT_SIZE = 3

# Loading the model

In [13]:
model = keras.models.load_model('../../../models/test/best_model_FRA40.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 512)            │     1,099,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,311,669 (12.63 MB)

 Trainable params: 1,103,889 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,207,780 (8.42 MB)

# Loading the previous sequences

In [14]:
base_df = pd.read_csv('../../../FRA40_H1_202403_2025.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
# base_df = pd.read_csv('../../../EURUSD_H1_202403_2025.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
base_df.rename(columns={
    '<OPEN>': 'OPEN', 
    '<HIGH>': 'HIGH', 
    '<LOW>': 'LOW', 
    '<CLOSE>': 'CLOSE'
}, inplace=True)

base_df = create_ts_from_df(base_df, '<DATE>', '<TIME>', 'DATETIME')
base_df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-03-07 19:00:00,8113.7,8173.0,8101.6,8148.7
2025-03-07 20:00:00,8149.2,8188.7,8148.6,8188.2
2025-03-07 21:00:00,8187.7,8206.7,8176.7,8185.6
2025-03-07 22:00:00,8185.1,8203.7,8168.6,8189.3
2025-03-07 23:00:00,8195.8,8196.8,8176.8,8190.3


In [15]:
df = base_df.loc[:'2025-03-07 10:00:00']
df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-03-07 06:00:00,8169.3,8173.8,8164.3,8165.8
2025-03-07 07:00:00,8165.3,8167.8,8149.3,8152.8
2025-03-07 08:00:00,8152.8,8153.8,8135.8,8136.8
2025-03-07 09:00:00,8137.8,8179.7,8133.8,8169.6
2025-03-07 10:00:00,8173.6,8178.2,8091.7,8107.7


# Generate the next close price 

In [16]:
dataset = TreadingDataset(df[['CLOSE']])
dataset.prepare_data()

In [17]:
last_prev_lines = dataset.feature_sequence[-1]
tk = PredictorToolkit(dataset.target_scaler, model, SEQUENCE_LENGTH)

for _ in range(5):
    last_prev_lines = dataset.feature_sequence[-1]
    entry_price = dataset.df[['CLOSE']].iloc[-1:].values[0][0]
    new_df = tk.predict_the_next_price(dataset.df[['CLOSE']], entry_price, last_prev_lines)
    dataset.update_df(new_df)

dataset.calculate_atr(base_df, period=14)
tk.predict_df

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step


,ENTRY_PRICE,PREDICTED_CLOSE
2025-03-07 11:00:00,8107.700000,8140.788574
2025-03-07 12:00:00,8140.788574,8172.411133
2025-03-07 13:00:00,8172.411133,8116.958008
2025-03-07 14:00:00,8116.958008,8144.250977
2025-03-07 15:00:00,8144.250977,8175.488281


# Make the trade strategy

## Stratégie avec un ratio perte/gain (Ratio Risk/Reward) de 1:2

perte de 10 pips pour gagner 20 pips
- Stop-loss pips = 0.0010
- Take-profit pips (Stop-loss pips * 2) = 0.0020


In [ ]:
dataset.df.tail()

,CLOSE,EMA,RSI,MACD,SIGNAL_LINE,UPPER_BAND,LOWER_BAND,BOLLINGER_PERCENT_B,ATR
2025-03-07 11:00:00,8140.788574,8160.687398,44.684452,-12.456699,-10.111561,8200.447695,8120.611162,0.252734,31.171429
2025-03-07 12:00:00,8172.411133,8161.803944,52.835088,-9.963488,-10.081947,8200.936159,8123.673812,0.630803,30.242857
2025-03-07 13:00:00,8116.958008,8157.532902,44.481802,-12.320185,-10.529594,8203.359757,8116.286015,0.007718,29.785714
2025-03-07 14:00:00,8144.250977,8156.267957,45.499387,-11.848982,-10.793472,8195.292639,8118.298230,0.337073,29.500000
2025-03-07 15:00:00,8175.488281,8158.098464,50.825838,-8.852910,-10.405360,8196.945120,8118.184577,0.727569,32.771429


In [19]:
strategy = TradeStrategy(tk.predict_df, 'CAC40', 2)
strategy.run(dataset.df)
strategy.df

,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT
2025-03-07 11:00:00,8107.700000,8140.788574,BUY,8130.788574,8160.788574
2025-03-07 12:00:00,8140.788574,8172.411133,BUY,8157.411133,8202.411133
2025-03-07 13:00:00,8172.411133,8116.958008,SELL,8126.958008,8096.958008
2025-03-07 14:00:00,8116.958008,8144.250977,BUY,8134.250977,8164.250977
2025-03-07 15:00:00,8144.250977,8175.488281,BUY,8160.488281,8205.488281


# Backtesting du model

In [20]:
backtest_data = base_df[['CLOSE']].merge(
    strategy.df, left_index=True, right_index=True
    ).merge(dataset.df[['ATR']], left_index=True, right_index=True)
backtest_data.merge(base_df[['HIGH', 'LOW']], left_index=True, right_index=True, how='inner')

,CLOSE,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT,ATR,HIGH,LOW
2025-03-07 11:00:00,8138.7,8107.700000,8140.788574,BUY,8130.788574,8160.788574,31.171429,8150.7,8096.0
2025-03-07 12:00:00,8108.5,8140.788574,8172.411133,BUY,8157.411133,8202.411133,30.242857,8139.7,8103.7
2025-03-07 13:00:00,8116.5,8172.411133,8116.958008,SELL,8126.958008,8096.958008,29.785714,8128.7,8102.0
2025-03-07 14:00:00,8105.2,8116.958008,8144.250977,BUY,8134.250977,8164.250977,29.500000,8126.2,8092.2
2025-03-07 15:00:00,8078.9,8144.250977,8175.488281,BUY,8160.488281,8205.488281,32.771429,8131.2,8073.7


In [21]:
backtester = Backtester(backtest_data)
backtester.simulate_trades()
performance_metrics = backtester.calculate_performance_metrics()

print("Métriques de performance :")
for key, value in performance_metrics.items():
    print(f"{key}: {value}")

Métriques de performance :
total_trades: 5
winning_trades: 5
losing_trades: 0
win_rate: 1.0
net_profit: 137.06396484375
average_profit: 27.41279296875
sharpe_ratio: 1.603694157902502


In [22]:
# Test de différents multiplicateurs
multipliers = [(1.0, 1.5), (1.5, 2.0), (2.0, 2.5)]

best_metrics = None
best_multipliers = None

for stop_loss_multiplier, take_profit_multiplier in multipliers:
    # Appliquez les multiplicateurs et recalculez les stop-loss et take-profit
    backtest_data['STOP_LOSS'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] + (stop_loss_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] - (stop_loss_multiplier * row['ATR']),
        axis=1
    )
    backtest_data['TAKE_PROFIT'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] - (take_profit_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] + (take_profit_multiplier * row['ATR']),
        axis=1
    )

    # Simule les trades et calcule les métriques
    backtester = Backtester(backtest_data)
    backtester.simulate_trades()
    metrics = backtester.calculate_performance_metrics()

    # Compare les métriques pour trouver les meilleurs paramètres
    if best_metrics is None or metrics['net_profit'] > best_metrics['net_profit']:
        best_metrics = metrics
        best_multipliers = (stop_loss_multiplier, take_profit_multiplier)

print("Meilleurs multiplicateurs :", best_multipliers)
print("Métriques associées :", best_metrics)

Meilleurs multiplicateurs : (1.0, 1.5)
Métriques associées : {'total_trades': 5, 'winning_trades': 3, 'losing_trades': 2, 'win_rate': 0.6, 'net_profit': 84.5496791294645, 'average_profit': 16.909935825892898, 'sharpe_ratio': 0.6549452102338501}
